# Kayak

## Get weather data with an API

In [125]:
import requests
import pandas as pd
import plotly.express as px
from statistics import fmean

In [126]:
cities_list = ["Mont Saint Michel",
               "St Malo",
               "Bayeux",
               "Le Havre",
               "Rouen",
               "Paris",
               "Amiens",
               "Lille",
               "Strasbourg",
               "Chateau du Haut Koenigsbourg",
               "Colmar",
               "Eguisheim",
               "Besancon",
               "Dijon",
               "Annecy",
               "Grenoble",
               "Lyon",
               "Gorges du Verdon",
               "Bormes les Mimosas",
               "Cassis",
               "Marseille",
               "Aix en Provence",
               "Avignon",
               "Uzes",
               "Nimes",
               "Aigues Mortes",
               "Saintes Maries de la mer",
               "Collioure",
               "Carcassonne",
               "Ariege",
               "Toulouse",
               "Montauban",
               "Biarritz",
               "Bayonne",
               "La Rochelle"]

### Get GPS coordinates

In [133]:
nominatim_payload = {
    'city':'',
    'country':'France',
    'format':'json'
}

custom_headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
}

df_cities = pd.DataFrame(columns=["id", "city", "lat", "lon"])

for index, element in enumerate(cities_list):
    nominatim_payload['city'] = element

    city = requests.get("http://nominatim.openstreetmap.org/search", params=nominatim_payload, headers=custom_headers).json()

    row = [index+1, element, float(city[0]["lat"]), float(city[0]["lon"])]

    df_cities = pd.concat([pd.DataFrame([row], columns=df_cities.columns), df_cities], ignore_index=True)

/var/folders/yt/qrbwlc0x6fj2rs4chrxkrm_m0000gn/T/ipykernel_55785/3928216839.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [134]:
display(df_cities)
display(df_cities.dtypes)

,id,city,lat,lon
0,35,La Rochelle,46.159732,-1.151595
1,34,Bayonne,43.494514,-1.473666
2,33,Biarritz,43.483252,-1.559278
3,32,Montauban,44.017584,1.354999
4,31,Toulouse,43.604462,1.444247
5,30,Ariege,42.845802,1.606759
6,29,Carcassonne,43.213036,2.349107
7,28,Collioure,42.525050,3.083155
8,27,Saintes Maries de la mer,43.451592,4.427720
9,26,Aigues Mortes,43.566152,4.191540


id       object
city     object
lat     float64
lon     float64
dtype: object

### Get weather informations about cities

In [135]:
open_weather_payload = {
    'lat':'',
    'lon':'',
    #'exclude':'current,minutely,hourly,alerts',
    'units':'metric',
    'appid':'1c442712fe2c9ef62a5041206e57d3ba'
}

list_mean_temp = []
list_mean_pop = []

for lat, lon in zip(df_cities["lat"], df_cities["lon"]):

    open_weather_payload['lat'] = lat
    open_weather_payload['lon'] = lon

    city_weather = requests.get("http://api.openweathermap.org/data/2.5/forecast", params=open_weather_payload).json()

    list_daily_temp = []
    list_pop = []

    for e in city_weather["list"]:
        list_daily_temp.append(e["main"]["temp"])
        list_pop.append(e["pop"])

    list_mean_temp.append(fmean(list_daily_temp))
    list_mean_pop.append(fmean(list_pop))

df_cities["mean_temp"] = list_mean_temp
df_cities["mean_pop"] = list_mean_pop

### Get top-5 destination

In [138]:
df_top_5_destinations = df_cities.sort_values(by=['mean_temp', 'mean_pop'], ascending=[False, True])[:5]
display(df_top_5_destinations)

,id,city,lat,lon,mean_temp,mean_pop
14,21,Marseille,43.296174,5.369953,13.74450,0.34675
2,33,Biarritz,43.483252,-1.559278,13.35325,0.44100
7,28,Collioure,42.525050,3.083155,13.26450,0.37375
1,34,Bayonne,43.494514,-1.473666,13.19175,0.43250
8,27,Saintes Maries de la mer,43.451592,4.427720,13.03750,0.25700


### Create CSV file

In [142]:
export_path = "/Users/qxzjy/vscworkspace/dsfs-ft-34/dcm_module/project/data/"
df_cities.to_csv(f"{export_path}cities.csv", index=False)

### Display best destinations map

In [141]:
fig = px.scatter_mapbox(df_top_5_destinations, lat="lat", lon="lon", color="mean_temp", size="mean_temp", mapbox_style="carto-positron")
fig.show()